In [44]:
from torchvision import datasets
from torchvision.transforms import ToTensor

In [46]:
# train_data = datasets.MNIST(
#     root = "data",
#     train = True,
#     transform = ToTensor(),
#     download = True 
# )
# test_data = datasets.MNIST(
#     root = 'data',
#     train = False,
#     transform = ToTensor(),
#     download = True 
# )


import torch
from torchvision import transforms
from torchvision.datasets import CSVDataset
from torch.utils.data import DataLoader

# Define transformations
transform = transforms.Compose([transforms.ToTensor()])

# Define the path to your CSV files
train_csv_path = "train.csv"
test_csv_path = "test.csv"

# Load CSV datasets
train_dataset = CSVDataset(train_csv_path, transform=transform)
test_dataset = CSVDataset(test_csv_path, transform=transform)

# Define batch size for DataLoader
batch_size = 64

# Create DataLoader for training and testing
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


ImportError: cannot import name 'CSVDataset' from 'torchvision.datasets' (C:\Program Files\Python311\Lib\site-packages\torchvision\datasets\__init__.py)

In [41]:
train_data

'train.csv'

In [42]:
test_data

'test.csv'

In [43]:
train_data.data.shape

AttributeError: 'str' object has no attribute 'data'

In [ ]:
test_data.data.shape

In [ ]:
train_data.targets

In [ ]:
# from torch.utils.data import DataLoader

# loaders = {
#     "train": DataLoader(train_data,
#                         batch_size=100,
#                         shuffle=True,
#                         num_workers=1),
    
#     "test": DataLoader(test_data,
#                        batch_size=100,
#                        shuffle=True,
#                        num_workers=1)
# }



import pandas as pd
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.data = pd.read_csv(csv_file)
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image = torch.tensor(self.data.iloc[idx, 0], dtype=torch.float32)  # Assuming the first column is the image
        label = torch.tensor(self.data.iloc[idx, 1], dtype=torch.long)     # Assuming the second column is the label

        if self.transform:
            image = self.transform(image)

        return image, label

# Example usage
custom_dataset = CustomDataset("train.csv", transform=transforms.ToTensor())
custom_loader = DataLoader(custom_dataset, batch_size=batch_size, shuffle=True)


In [ ]:
loaders

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)  # Flatten the tensor for the fully connected layer
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)




In [ ]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Create an instance of the CNN model
model = CNN().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.01)
loss_fn = nn.CrossEntropyLoss()

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(loaders['train']):
        data, target = data.to(device), target.to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        output = model(data)

        # Calculate the loss
        loss = loss_fn(output, target)

        # Backward pass
        loss.backward()

        # Update weights
        optimizer.step()

       # Print training progress
        if batch_idx % 20 == 0:
            print(f"Train Epoch: {epoch} [{ batch_idx * len(data)}/{len(loaders['train'].dataset)} ({100. * batch_idx / len(loaders['train']):.0f}%)]\t{loss.item():.6f}")

# Assuming you have a DataLoader named train_loader for your training data
# and log_interval is a variable indicating how often to print training progress.
# You'll need to set these values accordingly.

def test():
    model.eval()
    test_loss = 0
    correct = 0

    with torch.no_grad():
        for data, target in loaders['test']:  # Assuming you have a DataLoader named test_loader
            data, target = data.to(device), target.to(device)

            # Forward pass
            output = model(data)

            # Calculate the test loss
            test_loss += loss_fn(output, target).item()

            # Get the index of the max log-probability (predicted class)
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    # Average test loss
    test_loss /= len(loaders['test'].dataset)

    print(f"\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(loaders['test'].dataset)} ({100.*correct/len(loaders['test'].dataset):.0f}%\n")


In [ ]:
for epoch in range(1,11):
    train(epoch)
    test()

In [ ]:
device

In [ ]:
import matplotlib.pyplot as plt
model.eval()
data,target = test_data[2]
data = data.unsqueeze(0).to(device)
output = model(data)
prediction = output.argmax(dim=1,keepdim = True).item()
print(f"Prediction: {prediction}")
image = data.squeeze(0).squeeze(0).cpu().numpy()
plt.imshow(image, cmap='gray')
plt.show()